In [2]:
import pandas as pd

In [3]:
# Define file pathname
base = "/Users/ziadharmanani/Documents/GitHub/INF1340-Ridership/Ridership-Data/Bike share ridership "
extension = [
    "2023-01", "2023-02", "2023-03", "2023-04", "2023-05", "2023-06",
    "2023-07", "2023-08", "2023-09", "2023-10", "2023-11", "2023-12",
    ]

# Define columns order in a list
columns = [
    "Trip Id", "Trip Duration", "Start Station Id", "Start Time",
    "Start Station Name", "End Station Id", "End Time",
    "End Station Name", "Bike Id", "User Type"
    ]

# Used to store file paths
paths = []

# Loop through file extensions and combine them to locate them individually
for month in extension:
    path = base + month + ".csv"
    paths.append(path)

# Used to store the individual data frames
all_dfs = []

# Clean columns (Inconsistencies between 2024 and other years)
# Mismatch beteen the years issued a lot of NaN values. Trip Id was duplicated (2024 had its own).
for file in paths:
    df_temp = pd.read_csv(file, encoding = "latin1")
    df_temp.columns = [col.replace("ï»¿", "").strip() for col in df_temp.columns]

    # Used to clean BOM character for "Trip Id" (2024)
    if "ï»¿Trip Id" in df_temp.columns:
        df_temp["Trip Id"] = df_temp["ï»¿Trip Id"]
        df_temp = df_temp.drop(columns = ["ï»¿Trip Id"])

    #remove extra space in Trip Duration column
    #df.columns = df.columns.str.replace("Trip  Duration", "Trip Duration")
    df_temp.rename(columns={"Trip  Duration": "Trip Duration"}, inplace=True)

    # Reindexes the data frame according to columns list.
    df_temp = df_temp.reindex(columns = columns)
    all_dfs.append(df_temp)

# Concatinate dataframe
df = pd.concat(all_dfs, ignore_index = True)

# Drop "Model" to stay consistent with previous years (Only present in 2024)
df = df.drop(columns = ["Model", "Bike Id"], errors = "ignore")


df = df.set_index("Trip Id")

# NEXT STEPS: Look into Start Station Name and End Station Name.


In [4]:
df.columns.tolist()

['Trip Duration',
 'Start Station Id',
 'Start Time',
 'Start Station Name',
 'End Station Id',
 'End Time',
 'End Station Name',
 'User Type']

In [5]:
print(f"Start Station Name: {df['Start Station Name'].nunique()}")
print(f"End Station Name: {df['End Station Name'].nunique()}")

Start Station Name: 593
End Station Name: 592


In [6]:
df.isna().sum()

Trip Duration              0
Start Station Id           0
Start Time                 0
Start Station Name    595075
End Station Id          2944
End Time                   0
End Station Name      598563
User Type                  0
dtype: int64

In [7]:
#split start time into 2 columns
df[['Start Date', 'Start Time']] = df['Start Time'].str.split(' ', expand=True)

#move start date to the 3rd index
col = df.pop('Start Date')
df.insert(3, col.name, col)

#convert date to datetime format
df['Start Date'] = pd.to_datetime(df['Start Date'], format = '%m/%d/%Y')

In [8]:
#split date into day/month but keep og column
df['Start Day'] = df['Start Date'].dt.day

df['Start Month'] = df['Start Date'].dt.month

#move start day to the 4th index
col = df.pop('Start Day')
df.insert(4, col.name, col)

#move start month to the 5th index
col = df.pop('Start Month')
df.insert(5, col.name, col)


#change trip duration to minutes/hours
def convert(seconds):
    seconds = seconds % (24 * 60 * 60)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60

    return "%d:%02d:%02d" % (hour, minutes, seconds)


#df['Trip Duration'] = df['Trip Duration'].apply(convert)
df.tail()

,Trip Duration,Start Station Id,Start Time,Start Date,Start Day,Start Month,Start Station Name,End Station Id,End Time,End Station Name,User Type
Trip Id,,,,,,,,,,,
26682732,185,7391,23:56,2023-12-31,31,12,Yonge St / Dundas Sq,7041.0,12/31/2023 23:59,Edward St / Yonge St,Casual Member
26682733,802,7366,23:56,2023-12-31,31,12,Fort York Blvd / Bathurst St SMART,7048.0,01/01/2024 00:10,Front St W / Yonge St (Hockey Hall of Fame),Casual Member
26682735,1314,7203,23:57,2023-12-31,31,12,Bathurst St/Queens Quay(Billy Bishop Airport),7719.0,01/01/2024 00:19,NaN,Casual Member
26682737,371,7788,23:58,2023-12-31,31,12,NaN,7788.0,01/01/2024 00:05,NaN,Casual Member
26682738,1271,7298,23:59,2023-12-31,31,12,Bathurst St / Adelaide St W,7075.0,01/01/2024 00:21,Queens Quay W / Dan Leckie Way,Casual Member


In [9]:
#create weekday/weekend column
df['Weekday/Weekend'] = df['Start Date'].dt.weekday.apply(lambda x: 'Weekday' if x < 5 else 'Weekend')

df.head()

,Trip Duration,Start Station Id,Start Time,Start Date,Start Day,Start Month,Start Station Name,End Station Id,End Time,End Station Name,User Type,Weekday/Weekend
Trip Id,,,,,,,,,,,,
20148784,840,7022,00:00,2023-01-01,1,1,Simcoe St / Queen St W,7703.0,01/01/2023 00:14,NaN,Casual Member,Weekend
20148785,722,7399,00:01,2023-01-01,1,1,Lower Jarvis / Queens Quay E,7533.0,01/01/2023 00:13,Housey St / Dan Leckie Way,Casual Member,Weekend
20148786,1054,7269,00:02,2023-01-01,1,1,Toronto Eaton Centre (Yonge St),7076.0,01/01/2023 00:20,York St / Queens Quay W,Annual Member,Weekend
20148790,1329,7721,00:04,2023-01-01,1,1,NaN,7685.0,01/01/2023 00:26,NaN,Casual Member,Weekend
20148791,1291,7721,00:04,2023-01-01,1,1,NaN,7685.0,01/01/2023 00:26,NaN,Casual Member,Weekend


In [10]:
# Checking if the Weekday/Weekend column is correct in July
df[(df['Weekday/Weekend'] == 'Weekend') & (df['Start Date'].dt.year == 2023) & (df['Start Date'].dt.month == 7)].sample(10)

,Trip Duration,Start Station Id,Start Time,Start Date,Start Day,Start Month,Start Station Name,End Station Id,End Time,End Station Name,User Type,Weekday/Weekend
Trip Id,,,,,,,,,,,,
23098540,1510,7687,15:28,2023-07-16,16,7,NaN,7524.0,07/16/2023 15:53,Lisgar Park,Casual Member,Weekend
22709660,1239,7016,18:15,2023-07-01,1,7,Bay St / Queens Quay W (Ferry Terminal),7569.0,07/01/2023 18:36,Toronto Inukshuk Park,Casual Member,Weekend
22900244,530,7248,04:31,2023-07-09,9,7,Baldwin Ave / Spadina Ave - SMART,7273.0,07/09/2023 04:40,Bay St / Charles St - SMART,Casual Member,Weekend
22888435,750,7388,16:36,2023-07-08,8,7,Broadview Ave / Danforth Ave,7724.0,07/08/2023 16:48,NaN,Casual Member,Weekend
23276074,2172,7176,17:45,2023-07-22,22,7,Bathurst St / Fort York Blvd,7515.0,07/22/2023 18:21,Amos Waites Park,Casual Member,Weekend
22919231,327,7787,19:01,2023-07-09,9,7,NaN,7076.0,07/09/2023 19:07,York St / Queens Quay W,Casual Member,Weekend
23497399,871,7002,20:09,2023-07-30,30,7,St. George St / Bloor St W,7321.0,07/30/2023 20:23,Front St W / University Ave (2),Casual Member,Weekend
22700001,358,7468,12:54,2023-07-01,1,7,Front St / Simcoe St,7260.0,07/01/2023 13:00,Spadina Ave / Adelaide St W,Casual Member,Weekend
23260266,4457,7202,09:49,2023-07-22,22,7,Queen St W / York St (City Hall),7646.0,07/22/2023 11:03,University Ave / Richmond St W,Casual Member,Weekend


In [ ]:
# Added a new column Trip Duration (mm:ss)
df['Trip Duration (mm:ss)'] = df['Trip Duration'].apply(lambda x: f"{x // 60}:{x % 60:02d}")
df.sample(10)

,Trip Duration,Start Station Id,Start Time,Start Date,Start Day,Start Month,Start Station Name,End Station Id,End Time,End Station Name,User Type,Weekday/Weekend,Trip Duration (mm:ss)
Trip Id,,,,,,,,,,,,,
20989261,350,7322,22:50,2023-04-13,13,4,King St W / Brant St,7160.0,04/13/2023 22:56,King St W / Tecumseth St,Annual Member,Weekday,5:50
25549290,787,7149,11:34,2023-10-13,13,10,Yarmouth Rd / Christie St,7007.0,10/13/2023 11:48,College St / Huron St,Casual Member,Weekday,13:07
20201943,805,7433,08:19,2023-01-09,9,1,Dundas St E / Boulton Ave - SMART,7271.0,01/09/2023 08:33,Yonge St / Alexander St - SMART,Casual Member,Weekday,13:25
21485654,1940,7197,12:30,2023-05-13,13,5,Queen St W / Dovercourt Rd,7595.0,05/13/2023 13:02,Moore Park,Casual Member,Weekend,32:20
24330413,720,7542,16:02,2023-08-29,29,8,Queen St W / John St,7197.0,08/29/2023 16:14,Queen St W / Dovercourt Rd,Casual Member,Weekday,12:00
22807623,413,7788,18:07,2023-07-05,5,7,NaN,7076.0,07/05/2023 18:14,York St / Queens Quay W,Casual Member,Weekday,6:53
21683346,1208,7354,13:33,2023-05-22,22,5,Tommy Thompson Park (Leslie Street Spit),7344.0,05/22/2023 13:53,Cherry Beach,Annual Member,Weekday,20:08
24518473,578,7000,08:06,2023-09-05,5,9,Fort York Blvd / Capreol Ct,7033.0,09/05/2023 08:15,Union Station,Casual Member,Weekday,9:38
24477846,288,7131,16:46,2023-09-03,3,9,Taddle Creek Park,7140.0,09/03/2023 16:51,Macpherson Ave / Spadina Rd,Casual Member,Weekend,4:48
